In [1]:
import os
import json
import stanza
import csv
import pandas as pd

# Build an English pipeline
stanza.download('en', package='mimic', processors={'ner': 'i2b2'}) # download English model
nlp = stanza.Pipeline('en', package='mimic', processors={'ner': 'i2b2'}) # initialize English neural pipeline

C:\Users\okechukwu chude\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-03-19 23:51:18 INFO: Downloaded file to C:\Users\okechukwu chude\stanza_resources\resources.json
2024-03-19 23:51:18 INFO: Downloading these customized packages for language: en (English)...
| Processor       | Package        |
------------------------------------
| tokenize        | mimic          |
| pos             | mimic_charlm   |
| lemma           | mimic_nocharlm |
| depparse        | mimic_charlm   |
| ner             | i2b2           |
| pretrain        | mimic          |
| forward_charlm  | mimic          |
| backward_charlm | mimic          |

2024-03-19 23:51:18 INFO: File exists: C:\Users\okechukwu chude\stanza_resource

In [2]:
# Function to process text using Stanza
def process_text_with_stanza(text):
    doc = nlp(text)
    return doc

In [3]:
# Global variable to store all extracted texts
extracted_texts = []

In [4]:
def extract_info_from_json(json_file_path, hadm_id_set):
    global extracted_texts

    # Open the JSON file
    with open(json_file_path, 'r') as file:
        # Load JSON data from the file
        data = json.load(file)

        # Extract 'hadm_id' and 'comment' from the JSON data
        hadm_id = data.get('hadm_id', None)
        comment = data.get('comment', None)

        # If 'hadm_id' is not found, print a warning message
        if hadm_id is None:
            print(f"Warning: 'hadm_id' not found in {json_file_path}")
            return None

        # Add 'hadm_id' to the set
        hadm_id_set.add(hadm_id)

        # Extract 'notes' from the JSON data
        notes = data.get('notes', [])

        # Iterate through each note
        for note in notes:
            note_info = {}  # Initialize a dictionary to store note information

            # Add 'hadm_id' to the note information
            note_info['hadm_id'] = hadm_id

            # Extract information from the note
            note_info['note_id'] = note.get('note_id', None)
            note_info['category'] = note.get('category', None)
            note_info['description'] = note.get('description', None)

            # Extract annotations from the note
            annotations = note.get('annotations', [])
            annotations_info = []  # Initialize a list to store annotation information

            # Iterate through each annotation in the note
            for annotation in annotations:
                annotation_info = {}  # Initialize a dictionary to store annotation information

                # Extract information from the annotation
                annotation_info['begin'] = annotation.get('begin', None)
                annotation_info['end'] = annotation.get('end', None)
                annotation_info['code'] = annotation.get('code', None)
                annotation_info['code_system'] = annotation.get('code_system', None)
                annotation_info['description'] = annotation.get('description', None)
                annotation_info['type'] = annotation.get('type', None)
                annotation_info['covered_text'] = annotation.get('covered_text', None)

                annotations_info.append(annotation_info)  # Append annotation information to the list

            note_info['annotations'] = annotations_info  # Add annotations information to the note
            note_info['text'] = note.get('text', None)

            # Process text with Stanza
            if note_info['text']:
                processed_text = process_text_with_stanza(note_info['text'])
                note_info['processed_text'] = processed_text

                # Extract sentence information and generate labels
                sentence_info = []
                for sent in processed_text.sentences:
                    tokens = [word.text for word in sent.words]
                    labels = ['O'] * len(tokens)  # Initialize labels as 'O'

                    for annotation in note_info['annotations']:
                        begin = annotation['begin']
                        end = annotation['end']

                        word_indices = find_word_indices(sent.words, begin, end)

                        # Assign labels
                        if word_indices:
                            start_idx, end_idx = word_indices[0], word_indices[-1]
                            labels[start_idx] = 'B'
                            for idx in range(start_idx + 1, end_idx + 1):
                                labels[idx] = 'I'
                        # for idx in word_indices:
                        #     if sent.words[idx].start_char == begin:
                        #         labels[idx] = 'B'
                        #     elif sent.words[idx].start_char > begin and sent.words[idx].end_char < end:
                        #         labels[idx] = 'I'

                    sentence_info.append({
                        'sentence_id': sent.index,
                        'words': tokens,
                        'labels': labels
                    })

                    # Print the result for each sentence
                    print('Sentence:', ' '.join(tokens))
                    print('Labels:', ' '.join(labels))
                    print()

                note_info['sentence_info'] = sentence_info

            extracted_texts.append(note_info)  # Append note information to the global variable

#

In [5]:
def find_word_indices(words, begin, end):
    # Create an empty list to store the indices of words found within the given range
    word_indices = []

    # Iterate through each token in the list of tokens
    for idx, word in enumerate(words):
        # Check if the start character of the token matches the beginning of the range
        # or if the token spans the beginning of the range
        if word.start_char == begin or (word.start_char < begin and word.end_char > begin):
            # If it matches, add the index of the token to the list of word indices
            word_indices.append(idx)
        # Check if the end character of the token matches the end of the range
        # or if the token spans the end of the range
        if word.end_char == end or (word.start_char < end and word.end_char > end):
            # If it matches, add the index of the token to the list of word indices
            word_indices.append(idx)
            break

    # Return the list of word indices found within the given range
    return word_indices

In [6]:
def generate_labels(words, annotations, start_token_idx, end_token_idx):
    labels = ['O'] * len(words)

    for annotation in annotations:
        code = annotation['code']

        # # Update labels for the matched words
        # if start_token_idx is not None and end_token_idx is not None:
        #     labels[start_token_idx] = f'B-{code}'
        #     for idx in range(start_token_idx + 1, end_token_idx + 1):
        #         labels[idx] = f'I-{code}'

    return labels

In [7]:
# Function to search for JSON files in a given folder and its subfolders

def search_files(folder_path):
    # Set to store unique hadm_id values
    hadm_id_set = set()

    # Recursively search for JSON files in the folder and its subfolders
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if filename.endswith('.json'):
                # Construct the full path to the JSON file
                json_file_path = os.path.join(root, filename)
                print("Processing:", json_file_path)
                # Call extract_info_from_json function to extract information from the JSON file
                extract_info_from_json(json_file_path, hadm_id_set)
                # Print a separator after processing each file
                print("=" * 50)

    # Print the count of unique hadm_id values
    print("Total unique hadm_id count:", len(hadm_id_set))



In [8]:
# Specify the path to the main folder containing subfolders with JSON files
main_folder_path = r"C:\Users\okechukwu chude\Documents\NLP\text extraction\Automating-Medical-Coding\2"

# Call the search_files function to start searching for JSON files in the main folder and its subfolders
search_files(main_folder_path)

Processing: C:\Users\okechukwu chude\Documents\NLP\text extraction\Automating-Medical-Coding\2\100197-ICD-9.json


Sentence: Admission Date : [ **2136-10-23 **] Discharge Date : [ **2136-10-24 **]
Labels: O O O O O O O O O O O O

Sentence: Date of Birth : [ ** 2056-7-14 **] Sex : M
Labels: O O O O O O O O O O O

Sentence: Service : NEUROSURGERY
Labels: O O O

Sentence: Allergies : No Known Allergies / Adverse Drug Reactions
Labels: O O O O O O O O O

Sentence: Attending :[** First Name3 ( LF ) 1835 **] Chief Complaint : Found down
Labels: O O O O O O O O O O O O O O

Sentence: Major Surgical or Invasive Procedure : None
Labels: O O O O O O O

Sentence: History of Present Illness : 80M p/w a large R IPH with intraventricular extension , midline shift , and hydrocephalus .
Labels: O O O O O O O O O O B O B I O O O O O B O

Sentence: The patient was found lying face up in his bathtub after having shaved .
Labels: O O O O O O O O O O O O O O

Sentence: There was no water in the tub and the shower was not turned on .
Labels: O O O O O O O O O O O O O O O

Sentence: He was found by a neighbor .
Labels: O

In [9]:
import pandas as pd

# Function to create a DataFrame from the extracted sentence information
def create_dataframe(extracted_texts):
    data = []

    for note_info in extracted_texts:
        if 'sentence_info' in note_info:
            for sentence_info in note_info['sentence_info']:
                words = sentence_info['words']
                labels = sentence_info['labels']

                for word, label in zip(words, labels):
                    data.append({
                        'hadm_id': note_info['hadm_id'],
                        'note_id': note_info['note_id'],
                        'sentence_id': sentence_info['sentence_id'],
                        'word': word,
                        'label': label
                    })

    df = pd.DataFrame(data)
    return df

df = create_dataframe(extracted_texts)
print(df)

      hadm_id  note_id  sentence_id          word label
0      100197    25762            0     Admission     O
1      100197    25762            0          Date     O
2      100197    25762            0             :     O
3      100197    25762            0             [     O
4      100197    25762            0  **2136-10-23     O
...       ...      ...          ...           ...   ...
9868   101525   240524            2       tracing     O
9869   101525   240524            2     available     O
9870   101525   240524            2           for     O
9871   101525   240524            2    comparison     O
9872   101525   240524            2             .     O

[9873 rows x 5 columns]


In [10]:
label_counts = df['label'].value_counts()
print(label_counts)

label
O    9647
I     140
B      86
Name: count, dtype: int64


In [11]:
token_df= df.drop(['hadm_id', 'note_id',], axis=1)

In [12]:
token_df

,sentence_id,word,label
0,0,Admission,O
1,0,Date,O
2,0,:,O
3,0,[,O
4,0,**2136-10-23,O
...,...,...,...
9868,2,tracing,O
9869,2,available,O
9870,2,for,O
9871,2,comparison,O


In [13]:
token_df.rename(columns = {'word':'words', 'label':'labels'}, inplace = True)

In [14]:
token_df.head()

,sentence_id,words,labels
0,0,Admission,O
1,0,Date,O
2,0,:,O
3,0,[,O
4,0,**2136-10-23,O


In [15]:
x = token_df[['sentence_id', 'words']]
y = token_df['labels']

SPLITTING DATA

In [16]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3)

In [17]:
#build the training and test data
train_data = pd.DataFrame({'sentence_id': x_train['sentence_id'], 'words': x_train['words'], 'labels': y_train})
test_data = pd.DataFrame({'sentence_id': x_test['sentence_id'], 'words': x_test['words'], 'labels': y_test})

In [18]:
train_data

,sentence_id,words,labels
6116,26,18,O
8738,9,Fluid,O
3240,16,%,O
5835,16,",",O
6579,18,a,O
...,...,...,...
1313,8,was,O
9000,12,121,O
6731,5,EDH,B
3097,13,:,O


MODEL TRAINING

BERT

In [19]:
from simpletransformers.ner import NERModel, NERArgs

In [20]:
label = token_df['labels'].unique().tolist()
label

['O', 'B', 'I']

In [21]:
args = NERArgs()
args.num_train_epochs = 1
args.learning_rate = 1e-4
args.overwrite_output_dir = True
args.train_batch_size = 32
args.eval_batch_size = 32

In [22]:
model = NERModel('bert', 'bert-base-cased', labels=label, args=args, use_cuda = False)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
from sklearn.metrics import accuracy_score

model.train_model(train_data, eval_data = test_data, acc = accuracy_score)

Epoch 1 of 1: 100%|██████████| 1/1 [01:19<00:00, 79.00s/it]


(3, 0.8699794063965479)

In [24]:
result, model_outputs, preds_list = model.eval_model(test_data)

Running Evaluation: 100%|██████████| 3/3 [00:23<00:00,  7.80s/it]
C:\Users\okechukwu chude\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [25]:
result

{'eval_loss': 0.18093477189540863,
 'precision': 0.0,
 'recall': 0.0,
 'f1_score': 0.0}

In [26]:
prediction, model_output = model.predict(['There is\nalso intraventricular extension into the ipsilateral and\ncontralateral lateral ventricles.'])

Running Prediction: 100%|██████████| 1/1 [00:00<00:00,  2.84it/s]


In [27]:
prediction

[[{'There': 'O'},
  {'is': 'O'},
  {'also': 'O'},
  {'intraventricular': 'O'},
  {'extension': 'O'},
  {'into': 'O'},
  {'the': 'O'},
  {'ipsilateral': 'O'},
  {'and': 'O'},
  {'contralateral': 'O'},
  {'lateral': 'O'},
  {'ventricles.': 'O'}]]

BioBERT

In [28]:
# Create the NER model with BioBERT
model = NERModel('bert', 'dmis-lab/biobert-base-cased-v1.2', args=args, labels=label, use_cuda=False)

config.json: 100%|██████████| 1.11k/1.11k [00:00<?, ?B/s]
C:\Users\okechukwu chude\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\okechukwu chude\.cache\huggingface\hub\models--dmis-lab--biobert-base-cased-v1.2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-s

In [29]:
# Train the model
model.train_model(train_data)

# Evaluate the model on test data
result, model_outputs, preds_list = model.eval_model(test_data)

# Make predictions on new text
predictions, raw_outputs = model.predict(["There is also intraventricular extension into the ipsilateral and contralateral lateral ventricles."])
print(predictions)

Running Prediction: 100%|██████████| 1/1 [00:00<00:00,  2.60it/s]

[[{'There': 'O'}, {'is': 'O'}, {'also': 'O'}, {'intraventricular': 'O'}, {'extension': 'O'}, {'into': 'O'}, {'the': 'O'}, {'ipsilateral': 'O'}, {'and': 'O'}, {'contralateral': 'O'}, {'lateral': 'O'}, {'ventricles.': 'O'}]]


In [32]:
result

{'eval_loss': 0.14300517737865448,
 'precision': 0.0,
 'recall': 0.0,
 'f1_score': 0.0}

In [30]:

predictions, raw_outputs = model.predict(["T[**Name2 (NI) **] was extubated on [**10-24**] without"])
print(predictions)


Running Prediction: 100%|██████████| 1/1 [00:00<00:00,  3.67it/s]

[[{'T[**Name2': 'O'}, {'(NI)': 'O'}, {'**]': 'O'}, {'was': 'O'}, {'extubated': 'O'}, {'on': 'O'}, {'[**10-24**]': 'O'}, {'without': 'O'}]]
